In [1]:
from pprint import pprint

In [2]:
from data_load import load_mit_db
from utils import PrintTime

In [3]:
db_params = {
    "DS": None,
    "is_reduce": True,
    "do_preprocess":True,
    "ws": [90, 90],
}

ml_features_params={
    "maxRR": True,
    "use_RR": True,
    "norm_RR": True,
    "leads_flag": [1, 0],
    "compute_morph": ['wvlt', 'HOS', 'lbp', 'OurMorph'],
    "DS": "DS1",  # for wvlt+pca
    "ws": [90, 90],  # for mymorph
}


model_kwargs = {
    "c_value": 0.01,
    "gamma_value": 0.0,
    "multi_mode": "ovo"
    
}

In [4]:
db_trn_params = {}
db_trn_params.update(db_params)
db_trn_params["DS"] = "DS1"
ml_trn_features_params = ml_features_params
ml_trn_features_params = {}
ml_trn_features_params.update(ml_features_params)
ml_trn_features_params["DS"] = "DS1"
print(db_trn_params)

db_val_params = {}
db_val_params.update(db_params)
db_val_params["DS"] = "DS2"
ml_val_features_params = ml_features_params
ml_val_features_params = {}
ml_val_features_params.update(ml_features_params)
ml_val_features_params["DS"] = "DS2"
print(db_val_params)

load_trn_params = {}
load_trn_params.update(db_trn_params)
load_trn_params.update(ml_trn_features_params)
load_trn_params.pop("leads_flag")

load_val_params = {}
load_val_params.update(db_val_params)
load_val_params.update(ml_val_features_params)
load_val_params.pop("leads_flag")

pprint(load_trn_params)
pprint(load_val_params)

{'DS': 'DS1', 'is_reduce': True, 'do_preprocess': True, 'ws': [90, 90]}
{'DS': 'DS2', 'is_reduce': True, 'do_preprocess': True, 'ws': [90, 90]}
{'DS': 'DS1',
 'compute_morph': ['wvlt', 'HOS', 'lbp', 'OurMorph'],
 'do_preprocess': True,
 'is_reduce': True,
 'maxRR': True,
 'norm_RR': True,
 'use_RR': True,
 'ws': [90, 90]}
{'DS': 'DS2',
 'compute_morph': ['wvlt', 'HOS', 'lbp', 'OurMorph'],
 'do_preprocess': True,
 'is_reduce': True,
 'maxRR': True,
 'norm_RR': True,
 'use_RR': True,
 'ws': [90, 90]}


In [5]:
tr_features, tr_labels, tr_patient_num_beats = load_mit_db(**load_trn_params)
print(tr_features.shape)
print(tr_labels.shape)

Loading pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS1_rm_bsline_maxRR_RR_norm_RR_wvlt_HOS_lbp_OurMorph_reduced_MLII_V1.pkl...
(42617, 114)
(42617,)


In [6]:
eval_features, eval_labels, eval_patient_num_beats = load_mit_db(**load_val_params)
print(eval_features.shape)
print(eval_labels.shape)

Loading pickle: /home/congyu/dataset/ECG/mitdb/ml_learning/features/w_90_90_DS2_rm_bsline_maxRR_RR_norm_RR_wvlt_HOS_lbp_OurMorph_reduced_MLII_V1.pkl...
(42285, 114)
(42285,)


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()
scaler.fit(tr_features)
tr_features_scaled = scaler.transform(tr_features)
eval_features_scaled = scaler.transform(eval_features)

In [9]:
import numpy as np

In [10]:
from sklearn import svm
import time

In [11]:
verbose = False

C_value = model_kwargs.get("c_value", 1)
gamma_value = model_kwargs.get("gamma_value", 0)
multi_mode = model_kwargs.get("multi_model", "ovr")

# TODO load best params from cross validation!
use_probability = False

class_weights = {}
for c in range(4):
    class_weights.update({c: len(tr_labels) / float(np.count_nonzero(tr_labels == c))})

# class_weight='balanced',
if gamma_value != 0.0:  # NOTE 0.0 means 1/n_features default value
    svm_model = svm.SVC(C=C_value, kernel='rbf', degree=3, gamma=gamma_value,
                        coef0=0.0, shrinking=True, probability=use_probability, tol=0.001,
                        cache_size=200, class_weight=class_weights, verbose=verbose,
                        max_iter=-1, decision_function_shape=multi_mode, random_state=None)
else:
    svm_model = svm.SVC(C=C_value, kernel='rbf', degree=3, gamma='auto',
                        coef0=0.0, shrinking=True, probability=use_probability, tol=0.001,
                        cache_size=200, class_weight=class_weights, verbose=verbose,
                        max_iter=-1, decision_function_shape=multi_mode, random_state=None)



In [ ]:
with PrintTime("train a model"):
    svm_model.fit(tr_features_scaled, tr_labels)


working on train a model ...


In [ ]:
y_pred = svm_model.predict(eval_features_scaled)

In [ ]:
y_pred.shape

In [ ]:
from collections import Counter

In [ ]:
c = Counter(y_pred[0:10000])

In [ ]:
print(c)

In [ ]:
from sklearn.metrics import f1_score, classification_report
from config import AAMI_CLASSES

In [ ]:
print("marco f1 score", f1_score(eval_labels, y_pred, average='macro'))

In [ ]:
print(classification_report(eval_labels, y_pred, target_names=AAMI_CLASSES))